## MTH9878 Interest Rate Models HW7
* Group 05
* Author: Pan, Hongchao & Zhang, Chendi
* Kernel version: Python 2.7
* Packages: pandas, numpy, math, matplotlib, scipy, time, datetime, dateutil.relativedelta
* Data: Given DataSheetCurve data
* Notes:
    * The notes and steps to deduce formulas had been put at corresponding cells.
    * Build two classes from previous assignments to do simulation.
    * Notes of model build and simulation are added by Markdown.

### Q1: Implement the LIBOR market model

In [15]:
import math
import pandas as pd
import numpy as np
import time

from datetime import date
from datetime import timedelta
from scipy.interpolate import splev, splrep, splint
from dateutil.relativedelta import relativedelta

def diff_years(day_1,day_2):
    # to calculate the diff between two days
    diff_in_years = relativedelta(day_2, day_1).years
    diff_in_months = relativedelta(day_2, day_1).months
    diff_in_days = relativedelta(day_2, day_1).days
    return diff_in_years+diff_in_months/12.+diff_in_days/360.

### Build a LIBOR curve class and a swap class

In [18]:
import matplotlib.pyplot as plt
%matplotlib inline

# Define a Curve class for Libor/OIS rates
class Curve():
    # constructor
    def __init__(self, _df,_spot_time):
        """
        #  _df: input dataframe which contains data of rates and T
        """
        self.Ts  = _df['T']
        self.rates = _df['Rate']
        self.tck = splrep(self.Ts, self.rates)
        self.spot_time = _spot_time
    

    def disf(self,day_1,day_2):
        # function that computes the discount factor between any two dates
        # return the discount factor between day_1 and day_2
        # based on data from self.Ts and self.rates


        # get input time interval fraom day_1 and day_2
        T_begin = diff_years(self.spot_time,day_1)
        T_end = diff_years(self.spot_time,day_2)

        # calculate the discount factor
        disf = math.exp(-splint(T_begin, T_end, self.tck))

        return disf
    
    
    # To compute the forward LIBOR rate for any settlement and underlying tenor
    def forward_rate(self,S,T):
        # return the forward LIBOR rate for start S and maturity T (tenor T-S)
        # based on data from self.Ts and self.rates
        
        # get input time interval from day_1 and day_2
        T_begin = diff_years(self.spot_time,S)
        T_end = diff_years(self.spot_time,T)
        
        # calculate the forward rate
        forward_rate = (math.exp(splint(T_begin,T_end,self.tck))-1)/(T_end-T_begin)

        return forward_rate
    
    def dev(self,T,n):
        #to calculate nth degree derivative of the curve
        return splev(T,self.tck,der=n)

In [19]:
# Create a Swap class for swaps

class Swap():
    def __init__(self, _OIS_curve, _LIBOR_curve, _spot_time):
        self.OIS_curve = _OIS_curve
        self.LIBOR_curve = _LIBOR_curve
        self.spot_time = _spot_time
     

    # function that computes the (spot or forward) swap rate for any settlement and underlying tenor
    def swap_rate(self, _maturity_time, strike, start_year, libor_forward):
        # fixed leg payment
        fixed_leg = 0.
        maturity = _maturity_time
        while maturity > self.spot_time:
            fixed_leg += 0.5 * self.OIS_curve.disf(start_year,maturity+relativedelta(years=1))*strike
            maturity = maturity -  relativedelta(months=6)

        # floating leg payment
        maturity = _maturity_time
        float_leg = 0.
        while maturity > self.spot_time:
            float_leg += 0.25 * libor_forward[int(diff_years(_maturity_time, maturity)*4)-1]*ois.disf(start_year,maturity+relativedelta(years=1))
            maturity = maturity - relativedelta(months=3)
        
        return float_leg/fixed_leg, float_leg, fixed_leg
    
    # calculate the break-even basis rates
    def basis_rate(self, _maturity_time):
        # print(_maturity_time)
        maturity = _maturity_time
        # LIBOR leg
        LIBOR_leg = 0.
        while maturity > self.spot_time:
            LIBOR_leg += 0.25 * (self.LIBOR_curve.forward_rate(maturity-relativedelta(months=3),maturity)-self.OIS_curve.forward_rate(maturity-relativedelta(months=3),maturity))*self.OIS_curve.disf(self.spot_time,maturity)
            maturity = maturity -  relativedelta(months=3)
        
        # OIS and spread leg
        # print(_maturity_time)
        maturity = _maturity_time
        OIS_leg = 0.
        while maturity > self.spot_time:
            OIS_leg+= 0.25 *self.OIS_curve.disf(self.spot_time,maturity)
            maturity = maturity -  relativedelta(months=3)
       
        return LIBOR_leg/OIS_leg  

### Get data from the DataSheetCurve

In [20]:
import scipy.optimize as sco
from scipy import integrate


# target function that we want to minize
def min_func(rate):
    OIS_rates = rate[0:18].copy()
    LIBOR_rates = rate[18:].copy()
    
    Ts = [-15.0, -10.0, -5.0, 0.0, 1.0, 2.0, 3.0, 5.0, 7.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0]
    
    # get maturities
    spot_date = date(2011,12,15)
    OIS_dates = [spot_date +  relativedelta(months=int(T*12)) for T in Ts]
    LIBOR_dates = [spot_date +  relativedelta(months=int(T*12)) for T in Ts]
    
    #create temporary dataframe for OIS and LIBOR
    tmp_OIS_df = pd.DataFrame(OIS_rates,columns=['Rate'])
    tmp_OIS_df['T'] = Ts
    # print(tmp_OIS_df)
    
    tmp_LIBOR_df = pd.DataFrame(LIBOR_rates,columns=['Rate'])
    tmp_LIBOR_df['T'] = Ts
    
    # create class objects
    OIS_obj = Curve(tmp_OIS_df,spot_date)
    LIBOR_obj = Curve(tmp_LIBOR_df,spot_date)
    swap_obj = Swap(OIS_obj,LIBOR_obj,spot_date)
    
    # optimization goal function

    goal = sum(([swap_obj.basis_rate(spot_date + relativedelta(months = int(T*12.))) for T in Basis_Swap_Rates_df['T']]-Basis_Swap_Rates_df['Basis (bp)']*1e-4)**2)+\
           sum(([LIBOR_obj.forward_rate(spot_date + relativedelta(months = int(T*12.)),spot_date + relativedelta(months = 3) + relativedelta(months = int(T*12.))) for T in ED_Futures_df['T']]-ED_Futures_df['Rate'])**2)+\
           sum(([swap_obj.swap_rate(spot_date + relativedelta(months = int(T*12.))) for T in Swap_Rates_df['T']]-Swap_Rates_df['Rate'])**2)
      
    # penalty lambda = 0.05
    penalty  = 1./2. * 0.05*integrate.quad(lambda x:OIS_obj.dev(x,2)**2 + LIBOR_obj.dev(x,2)**2,0,30)[0]

    return goal/2. + penalty

In [21]:
curve_data = pd.read_excel('DataSheetCurve.xls',sheetname=1)

# LIBOR data
LIBOR_df = curve_data.iloc[1:3,[0,1,2,3]].copy()
LIBOR_df.columns = ['Index','Start Date','End Date','Rate']
LIBOR_df.set_index('Index',inplace=True)

# get time interval T in the data
LIBOR_df['T'] = [0.,0.25]

# ED Futures data
ED_Futures_df = curve_data.iloc[5:13,[0,1,2,3,4]].copy()
ED_Futures_df.columns = ['Index','IMM date','Price','Conv. Adj. (bp)','Rate']
ED_Futures_df.set_index('Index',inplace=True)

# get time interval T in the data
ED_Futures_df['T'] = [0.,0.25,0.5,0.75,1.,1.25,1.5,1.75]

# Swap Rates data
Swap_Rates_df = curve_data.iloc[15:26,[0,1,2,3]].copy()
Swap_Rates_df.columns = ['Index','Start Date','End Date','Rate']
Swap_Rates_df.set_index('Index',inplace=True)

# get time interval T in the data
Swap_Rates_df['T'] = [2.,3.,4.,5.,7.,10.,12.,15.,20.,25.,30.]

# Fed Funds data
Fed_Funds_df = curve_data.iloc[28:29,[0,1,2,3]].copy()
Fed_Funds_df.columns = ['Index','Start Date','End Date','Rate']
Fed_Funds_df.set_index('Index',inplace=True)

# get time interval T in the data
Fed_Funds_df['T'] = [0.]

# Basis Swap Rates data
Basis_Swap_Rates_df = curve_data.iloc[32:58,[0,1,2,3]].copy()
Basis_Swap_Rates_df.columns = ['Index','Start Date','End Date','Basis (bp)']
Basis_Swap_Rates_df.set_index('Index',inplace=True)

# get time interval T in the data
Basis_Swap_Rates_df['T'] = [0.25,0.5,0.75,1.,1.5,2.,3.,4.,5.,7.,10.,12.,15.,20.,25.,30.]

In [22]:
Ts = [-3.0,-2.0,-1.0,0.0,1.0,2.0,3.0,5.0,7.0,10.0,15.0,20.0,25.0,31.0,32.0,33.0,34.0,35.0]

LIBOR_rates = np.array([ 0.00948189,  0.01414475, -0.00917321,  0.00723922,  0.00633716,
        0.00864967,  0.01474671,  0.02575979,  0.03156454,  0.03448485,
        0.03298521,  0.03002743,  0.0298039 ,  0.02931985,  0.02442053,
        0.00629368,  0.01080132,  0.00989984])

In [23]:
OIS_rates = np.array([ 0.00948189,  0.01414475, -0.00917321,  0.00723922,  0.00633716,
        0.00864967,  0.01474671,  0.02575979,  0.03156454,  0.03448485,
        0.03298521,  0.03002743,  0.0298039 ,  0.02931985,  0.02442053,
        0.00629368,  0.01080132,  0.00989984])

### Implement the spectral decomposition method to simulate a Brownian motion

In [24]:
# spectral decomposition method
class spectral_decomposition_method:
    def __init__(self, dt, n, p):
        self.cov = np.ones((n,n))
        self.p = p
        
        for i in range(1,n+1):
            for j in range(1,n+1):
                self.cov[i-1,j-1] = np.minimum(i,j) * dt

        self.eigen_val, self.eigen_vec = np.linalg.eig(self.cov)
        
        # np.random.seed(19)
        self.wt = np.sum((np.sqrt(self.eigen_val) * self.eigen_vec.T * np.random.normal(size = n))[:p],axis = 0)
        self.wt = np.insert(self.wt, 0, 0.)
        self.dwt = np.diff(self.wt)
        # print(self.dwt)
    
    def get_value(self, t):
        return self.dwt[t]

### 1-factor LMM using Euler’s scheme

In [25]:
class LMM:
    def __init__(self, steps, periods, libor_0, frozen = False):
        self.steps = steps
        self.periods = periods
        self.libor = np.ones((periods,steps + 1))
        self.libor[:,0] = libor_0
        self.dt = 1./steps
        self.vol = 0.0085
        self.frozen = frozen
    
    def simulate(self):
        self.get_brownian = spectral_decomposition_method(self.dt, self.steps, self.steps)
        for i in range(self.steps):
            # dw = self.get_brownian.get_value()
            for j in range(self.periods):
                # get self.drift
                self.drift = 0
                if not self.frozen:
                    for k in range(j,self.periods):
                        self.drift = self.drift - 0.25 * (self.vol**2)/(1. + 0.25 * self.libor[k][i-1]) if i > 0 else self.drift - 0.25 * (self.vol**2)/(1. + 0.25 * self.libor[k][i])
                else:
                    for k in range(j,self.periods):
                        self.drift = self.drift - 0.25 * (self.vol**2)/(1. + 0.25 * self.libor[k][0])
                
                # update libor
                self.libor[j,i + 1] = np.max(self.libor[j,i] + self.drift*self.dt + self.vol*self.get_brownian.get_value(i),0)
                
        return self.libor[:,-1]
    
    def evaluation(self, ois, start_year, N_MC, maturity, strike):
        value_sum = 0.0
        for i in range(N_MC):
            libor_forward = self.simulate()
            # print(libor_forward)
            
            # get a swap
            T = np.linspace(1,40,40)
            libor_df = pd.DataFrame(np.array([T,libor_forward]).T, columns=['T','Rate'])
            libor = Curve(libor_df, start_year)
            swap = Swap(ois, libor, start_year - relativedelta(years=1))
            
            _temp_, float_, fixed_ = swap.swap_rate(maturity, strike, start_year, libor_forward)
            
            #payoff
            value_ = (fixed_ - float_) * 100.
            
            # print(value_, float_, fixed_)
            if value_ > 0:
                value_sum += value_
       
        simulated_value = value_sum*ois.disf(date(2000,1,1),start_year)/N_MC
        return simulated_value  

In [26]:
spot_date = date(2000,1,1)

libor_df = pd.DataFrame(np.array([Ts,LIBOR_rates]).T, columns=['T','Rate'])
ois_df = pd.DataFrame(np.array([Ts,OIS_rates]).T, columns=['T','Rate'])
libor = Curve(libor_df,spot_date)
ois = Curve(ois_df,spot_date)

In [27]:
# get LMM
steps = 30
periods = 40

# get initial value of libor
libor_0 = [libor.forward_rate(spot_date+relativedelta(years=(1+3*i//12))+relativedelta(month=3*i%12),
                                          spot_date+relativedelta(years=(1+3*(i+1)//12))+relativedelta(month=3*(i+1)%12)) for i in range(periods)]
libor_0 = np.array(libor_0)

#with or without frozen curve
LMM_model_1 = LMM(steps,periods,libor_0,frozen = True)
LMM_model_2 = LMM(steps,periods,libor_0,frozen = False)

### Q2 Simulation

### LMM exact calculation

In [28]:
# evaluate 1Y into 10Y European receiver swaption
import time
# exact calculation
start_time = time.time()
n = 2000
print('2000 simulatison: {}'.format(LMM_model_2.evaluation(ois, date(2001,1,1), n, maturity=date(2010,1,1), strike=0.03872)))
end_time = time.time()
print('Takes {} seconds.'.format(end_time - start_time))

n = 5000
start_time = time.time()
print('5000 simulatison: {}'.format(LMM_model_2.evaluation(ois, date(2001,1,1), n, maturity=date(2010,1,1), strike=0.03872)))
end_time = time.time()
print('Takes {} seconds.'.format(end_time - start_time))

2000 simulatison: 13.0257415649
Takes 132.153088093 seconds.
5000 simulatison: 13.0144942451
Takes 332.962799072 seconds.


### The results between 2000 and 5000 simulated paths are close. Calculation is accurate.

### Then we simulate LMM with frozen curve approximation

In [29]:
# frozen curve approximation
start_time = time.time()
n = 2000
print('2000 simulatison: {}'.format(LMM_model_1.evaluation(ois, date(2001,1,1), n, maturity=date(2010,1,1), strike=0.03872)))
end_time = time.time()
print('Takes {} seconds.'.format(end_time - start_time))

n = 5000
start_time = time.time()
print('5000 simulatison: {}'.format(LMM_model_1.evaluation(ois, date(2001,1,1), n, maturity=date(2010,1,1), strike=0.03872)))
end_time = time.time()
print('Takes {} seconds.'.format(end_time - start_time))

2000 simulatison: 13.0413159982
Takes 136.426939964 seconds.
5000 simulatison: 12.9919277063
Takes 344.23606801 seconds.


### Conclusion: The results between two drift term calculation methods are similar. And they are both accurate.